# Git Storage Model Jake's class

## Description
https://wiki/display/gqd/Git+Storage

In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

2021-05-17 14:29:02,278: matplotlib.pyplot: 290: DEBUG: Loaded backend module://ipykernel.pylab.backend_inline version unknown..


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import os, sys
path = os.getcwd()
while not path.endswith('research'):    
    path = os.path.abspath(os.path.join(path, '../'))
sys.path.append(path)

In [16]:
import os, sys
import pandas as pd
import numpy as np
from datetime import datetime, date

In [17]:
#a number of utilities:
from gmt.fom.gas.shared.utils.common import pa
from gmt.fom.gas.shared.utils.dframe import ddff
from gmt.fom.gas.shared.utils.common import flatten_lst

In [20]:
from gmt.fom.gas.shared.models.git_storage.git_storage import GitStorageModel

In [21]:
import cmdty_storage

In [22]:
from cmdty_storage import CmdtyStorage, RatchetInterp
import pandas as pd
storage_simple = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    min_inventory = 0.0,
    max_inventory = 1500.0,
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9
)

storage_with_ratchets = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    ratchets= [
                ('2021-04-01', # For days after 2021-04-01 (inclusive) until 2022-10-01 (exclusive):
                       [
                            (0.0, -150.0, 250.0),    # At min inventory of zero, max withdrawal of 150, max injection 250
                            (2000.0, -200.0, 175.0), # At inventory of 2000, max withdrawal of 200, max injection 175
                            (5000.0, -260.0, 155.0), # At inventory of 5000, max withdrawal of 260, max injection 155
                            (7000.0, -275.0, 132.0), # At max inventory of 7000, max withdrawal of 275, max injection 132
                        ]),
                  ('2022-10-01', # For days after 2022-10-01 (inclusive):
                       [
                            (0.0, -130.0, 260.0),    # At min inventory of zero, max withdrawal of 130, max injection 260
                            (2000.0, -190.0, 190.0), # At inventory of 2000, max withdrawal of 190, max injection 190
                            (5000.0, -230.0, 165.0), # At inventory of 5000, max withdrawal of 230, max injection 165
                            (7000.0, -245.0, 148.0), # At max inventory of 7000, max withdrawal of 245, max injection 148
                        ]),
                 ],
    ratchet_interp = RatchetInterp.LINEAR
)

In [23]:
from cmdty_storage import three_factor_seasonal_value

# Creating the Inputs
monthly_index = pd.period_range(start='2021-04-25', periods=25, freq='M')
monthly_fwd_prices = [16.61, 15.68, 15.42, 15.31, 15.27, 15.13, 15.96, 17.22, 17.32, 17.66, 
                      17.59, 16.81, 15.36, 14.49, 14.28, 14.25, 14.32, 14.33, 15.30, 16.58, 
                      16.64, 16.79, 16.64, 15.90, 14.63]
fwd_curve = pd.Series(data=monthly_fwd_prices, index=monthly_index).resample('D').fillna('pad')

In [24]:
rates = [0.005, 0.006, 0.0072, 0.0087, 0.0101, 0.0115, 0.0126]
rates_pillars = pd.PeriodIndex(freq='D', 
                               data=['2021-04-25', '2021-06-01', '2021-08-01', '2021-12-01', '2022-04-01', 
                                              '2022-12-01', '2023-12-01'])
ir_curve = pd.Series(data=rates, index=rates_pillars).resample('D').asfreq('D').interpolate(method='linear')

In [25]:
def settlement_rule(delivery_date):
    return delivery_date.asfreq('M').asfreq('D', 'end') + 20

# Call the three-factor seasonal model
three_factor_results = three_factor_seasonal_value(
    cmdty_storage = storage_with_ratchets,
    val_date = '2021-04-25',
    inventory = 1500.0,
    fwd_curve = fwd_curve,
    interest_rates = ir_curve,
    settlement_rule = settlement_rule,
    num_sims = 2000,
    seed = 12,
    spot_mean_reversion = 91.0,
    spot_vol = 0.85,
    long_term_vol =  0.30,
    seasonal_vol = 0.19,
    basis_funcs = '1 + x_st + x_sw + x_lt + s + x_st**2 + x_sw**2 + x_lt**2 + s**2 + s * x_st',
    discount_deltas = True
)

# Inspect the NPV results
print('Full NPV:\t{0:,.0f}'.format(three_factor_results.npv))
print('Intrinsic NPV: \t{0:,.0f}'.format(three_factor_results.intrinsic_npv))
print('Extrinsic NPV: \t{0:,.0f}'.format(three_factor_results.extrinsic_npv))

2021-05-17 14:29:15,237: cmdty.storage.multi-factor: 400: INFO: Compiling basis functions. Takes a few seconds on the first run..
2021-05-17 14:29:15,239: cmdty.storage.multi-factor: 402: INFO: Compilation of basis functions complete..
2021-05-17 14:29:15,240: cmdty.storage.multi-factor: 405: INFO: Calculating intrinsic value..
2021-05-17 14:29:15,718: cmdty.storage.multi-factor: 410: INFO: Calculation of intrinsic value complete..
2021-05-17 14:29:15,720: cmdty.storage.multi-factor: 413: INFO: Calculating LSMC value..
2021-05-17 14:29:15,723: cmdty.storage.multi-factor: 325: INFO: Starting regression spot price simulation..
2021-05-17 14:29:15,890: cmdty.storage.multi-factor: 325: INFO: Spot regression price simulation complete..
2021-05-17 14:29:15,892: cmdty.storage.multi-factor: 325: INFO: Starting backward induction..
2021-05-17 14:29:31,222: cmdty.storage.multi-factor: 325: INFO: Completed backward induction..
2021-05-17 14:29:31,223: cmdty.storage.multi-factor: 325: INFO: Starti

Full NPV:	78,175
Intrinsic NPV: 	40,976
Extrinsic NPV: 	37,199


In [ ]:
%matplotlib inline
ax_deltas = three_factor_results.deltas.plot(title='Daily Deltas vs Projected Inventory', legend=True, label='Delta')
ax_deltas.set_ylabel('Delta')
inventory_projection = three_factor_results.expected_profile['inventory']
ax_inventory = inventory_projection.plot(secondary_y=True, legend=True, ax=ax_deltas, label='Expected Inventory')
h1, l1 = ax_deltas.get_legend_handles_labels()
h2, l2 = ax_inventory.get_legend_handles_labels()
ax_inventory.set_ylabel('Inventory')
ax_deltas.legend(h1+h2, l1+l2, loc=1)

In [ ]:
%matplotlib inline
ax_triggers = three_factor_results.trigger_prices['inject_trigger_price'].plot(
    title='Trigger Prices vs Forward Curve', legend=True)
three_factor_results.trigger_prices['withdraw_trigger_price'].plot(legend=True)
fwd_curve['2021-04-25' : '2022-04-01'].plot(legend=True)
ax_triggers.legend(['Inject Trigger Price', 'Withdraw Trigger', 'Forward Curve'])

In [ ]:
from cmdty_storage import storage_gui as gui

In [ ]:
import pandas as pd

In [ ]:
pd

In [ ]:
import cmdty_storage as cs

In [ ]:
import storage_gui as sg

sg.display_gui()

# Market

## Forward

In [ ]:
from gmt.fom.gas.shared.models.git_storage.data.fwd import Forward

In [ ]:
forward = Forward(True,True,0.999)

In [ ]:
forward.params

In [ ]:
forward.curve.plot()

## IR

In [ ]:
from gmt.fom.gas.shared.models.git_storage.data.ir import IR

In [ ]:
ir = IR()

In [ ]:
ir.curve

# Storage

In [ ]:
from gmt.fom.gas.shared.models.git_storage.data.storage import Storage

In [ ]:
storage = Storage('SIMPLE', 
                  storage_start='01/04/2021',
                  storage_end='01/04/2022',
                  injection_cost=0.0,
                  withdrawal_cost=0.0,
                  cmdty_consumed_inject=0.0,
                  cmdty_consumed_withdraw=0.0,
                  inj_consumed_pctg=0.0,
                  with_consumes_pctg=0.0,
                  #Specific to SIMPLE type:
                  min_inventory=0.0,
                  max_inventory=0.0,
                  max_injection_rate=0.0,
                  max_withdrawal_rate=0.0)

In [ ]:
storage.storage_type

In [ ]:
storage.params

# Calibrator

In [ ]:
from gmt.fom.gas.shared.models.git_storage.calibrator import Calibrator

In [ ]:
calibrator = Calibrator(spot_mean_reversion=30.0,
                        spot_vol=0.51,
                        long_term_vol=0.31,
                        seasonal_vol=0.2,
                        num_sims=4000,
                        basis_funcs='1 + x_st + x_sw + x_lt + x_st ** 2 + x_sw ** 2 + x_lt ** 2 + x_st ** 3 + x_sw ** 3 + x_lt ** 3',
                        seed=11,
                        seed_is_random=False,
                        fwd_sim_seed=13,
                        set_fwd_sim_seed=True,
                        extra_decisions=0,
                        num_inventory_grid_points=100,
                        numerical_tolerance=1e-10)

In [ ]:
calibrator.params

# Model

In [ ]:
myModel = GitStorageModel(asof=date(2021, 1, 20),
                          inventory=0.0,
                          discount_deltas=False,
                          ir = ir,
                          forward=forward,
                          storage=storage, 
                          calibrator=calibrator
                          )
myModel.fit()
myModel.report()

In [ ]:
myModel.ir.curve

In [ ]:
myModel.forward.params

In [ ]:
myModel.forward.curve.plot()

In [ ]:
myModel.storage.params

In [ ]:
myModel.calibrator.params

## Let's try to compute

In [ ]:
myModel.forward.curve

In [ ]:
from cmdty_storage import CmdtyStorage, three_factor_seasonal_value, MultiFactorModel, multi_factor, RatchetInterp

In [ ]:
myModel.storage.storage_type.value

In [ ]:
fwd_curve = myModel.forward.curve

In [ ]:
storage = CmdtyStorage(freq='D',
                      storage_start=myModel.storage.storage_start,
                      storage_end=myModel.storage.storage_end,
                      injection_cost=myModel.storage.injection_cost,
                      withdrawal_cost=myModel.storage.withdrawal_cost,
                      min_inventory=myModel.storage.min_inventory,
                      max_inventory=myModel.storage.max_inventory,
                      max_injection_rate=myModel.storage.max_injection_rate,
                      max_withdrawal_rate=myModel.storage.max_withdrawal_rate,
                      cmdty_consumed_inject=myModel.storage.cmdty_consumed_inject,
                      cmdty_consumed_withdraw=myModel.storage.cmdty_consumed_withdraw)

In [ ]:
myModel.ir.curve

In [ ]:
def twentieth_of_next_month(period): return period.asfreq('M').asfreq('D', 'end') + 20

In [ ]:
pd.Period(myModel.storage.storage_end, freq='D')

In [ ]:
twentieth_of_next_month(pd.Period(myModel.storage.storage_end, freq='D'))

In [ ]:
myModel.storage.storage_end

In [ ]:
interest_rate_curve = pd.Series(index=pd.period_range(myModel.asof,
twentieth_of_next_month(
pd.Period(myModel.storage.storage_end, freq='D')), freq='D'),
dtype='float64')

In [ ]:
interest_rate_curve[:] = myModel.ir.curve

In [ ]:
interest_rate_curve

In [ ]:
seed = None if myModel.calibrator.seed_is_random else myModel.calibrator.seed

In [ ]:
fwd_sim_seed = myModel.calibrator.fwd_sim_seed if myModel.calibrator.fwd_sim_seed else None

In [ ]:
import gmt.fom.gas.shared.config

In [ ]:
import logging

In [ ]:
logger = logging.getLogger()

In [ ]:
def on_progress(progress):
    logger.info(progress)
    

In [ ]:
val_results_3f = three_factor_seasonal_value(
    cmdty_storage = storage,
    val_date = myModel.asof,
    inventory=myModel.inventory,
    fwd_curve=fwd_curve,
    interest_rates=interest_rate_curve,
    settlement_rule=twentieth_of_next_month,
    spot_mean_reversion=myModel.calibrator.spot_mean_reversion,
    spot_vol=myModel.calibrator.spot_vol,
    long_term_vol=myModel.calibrator.long_term_vol,
    seasonal_vol=myModel.calibrator.seasonal_vol,
    num_sims=myModel.calibrator.num_sims,
    basis_funcs=myModel.calibrator.basis_funcs,
    discount_deltas=myModel.discount_deltas,
    seed=seed,
    fwd_sim_seed=fwd_sim_seed,
    extra_decisions=myModel.calibrator.extra_decisions,
    num_inventory_grid_points=myModel.calibrator.num_inventory_grid_points,
    on_progress_update=on_progress,
    numerical_tolerance=myModel.calibrator.numerical_tolerance
)

In [ ]:
params_dict = dict(cmdty_storage = storage,
    val_date = myModel.asof,
    inventory=myModel.inventory,
    fwd_curve=fwd_curve,
    interest_rates=interest_rate_curve,
    settlement_rule=twentieth_of_next_month,
    spot_mean_reversion=myModel.calibrator.spot_mean_reversion,
    spot_vol=myModel.calibrator.spot_vol,
    long_term_vol=myModel.calibrator.long_term_vol,
    seasonal_vol=myModel.calibrator.seasonal_vol,
    num_sims=myModel.calibrator.num_sims,
    basis_funcs=myModel.calibrator.basis_funcs,
    discount_deltas=myModel.discount_deltas,
    seed=seed,
    fwd_sim_seed=fwd_sim_seed,
    extra_decisions=myModel.calibrator.extra_decisions,
    num_inventory_grid_points=myModel.calibrator.num_inventory_grid_points,
    on_progress_update=on_progress,
    numerical_tolerance=myModel.calibrator.numerical_tolerance)

In [ ]:
storage

In [ ]:
params_dict['settlement_rule']

In [ ]:
b = 5

gui.on_load_val_data_clicked(b)

gui.on_load_curve_params(b)

gui.on_import_fwd_curve_clicked(b)

gui.on_load_storage_details_clicked(b)

gui.on_load_vol_params_clicked(b)

gui.on_load_tech_params(b)

res = gui.btn_clicked(b)

# res_tpl = stb.utils.common.to_namedtuple(res, True)